In [53]:
import numpy as np

创建数据集：词条和标签向量

In [54]:
def loadDataSet():
    # 切分的词条
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    # 类别标签向量，1代表侮辱性词汇，0代表不是
    classVec = [0, 1, 0, 1, 0, 1]
    # 返回实验样本切分的词条、类别标签向量
    return postingList, classVec # 返回 切分词条后（去标点）的文档组合和 类别标签的集合

将样本词条整理为不重复单词的词条列表：词汇表

In [55]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document) # 求集合并集
    return list(vocabSet)

将词条转换为向量：
词集模型 AND 词袋模型

In [56]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList) # 元素全为0的向量
    for word in inputSet:
        if word in vocabList:
            returnVec [vocabList.index(word)]= 1
        else:
            print("the word : %s is not in vocabList" % word)
    return returnVec

In [ ]:
def bagofWords2VecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList) # 元素全为0的向量
    for word in inputSet:
        if word in vocabList:
            returnVec [vocabList.index(word)] += 1 # 统计出现次数
        else:
            print("the word : %s is not in vocabList" % word)
    return returnVec

用词汇表检查向量

In [57]:
# listOPosts, listClasses = loadDataSet()
# myVocabList = createVocabList(listOPosts)
# print(setOfWords2Vec(myVocabList, listOPosts[0])) # listOPosts[0] 代表第一行向量

训练算法：通过词向量计算概率

        改进：降低特殊情况的影响，所有词出现次数初始化为1，分母初始化为2，取对数防止下溢出

In [58]:
def trainNB0(trainMatrix, trainCategory):
    # 计算训练文档数目
    numTrainDocs = len(trainMatrix)
    # 计算每篇文档的词条数目
    numWords = len(trainMatrix[0])
    # 文档属于侮辱类的概率
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    # 创建numpy.zeros数组，词条出现数初始化为0
    # p0Num = np.zeros(numWords)
    # p1Num = np.zeros(numWords)
    # 创建numpy.ones数组，词条出现数初始化为1,拉普拉斯平滑
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)
    # 分母初始化为0
    # p0Denom = 0.0
    # p1Denom = 0.0
    # 分母初始化为2，拉普拉斯平滑
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        # 统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)...
        if trainCategory[i] == 1:
            # 统计所有侮辱类文档中每个单词出现的个数
            p1Num += trainMatrix[i]
            # 统计一共出现的侮辱单词的个数
            p1Denom += sum(trainMatrix[i])
        # 统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)...
        else:
            # 统计所有非侮辱类文档中每个单词出现的个数
            p0Num += trainMatrix[i]
            # 统计一共出现的非侮辱单词的个数
            p0Denom += sum(trainMatrix[i])
    # 每个侮辱类单词分别出现的概率
    # p1Vect = p1Num / p1Denom
    # 取对数，防止下溢出
    p1Vect = np.log(p1Num / p1Denom)
    # 每个非侮辱类单词分别出现的概率
    # p0Vect = p0Num / p0Denom
    # 取对数，防止下溢出
    p0Vect = np.log(p0Num / p0Denom)
    # 返回属于侮辱类的条件概率数组、属于非侮辱类的条件概率数组、文档属于侮辱类的概率
    return p0Vect, p1Vect, pAbusive

In [59]:
# listOPosts, listClasses = loadDataSet()
# myVocabList = createVocabList(listOPosts)
# trainMat = []
# for postinDoc in listOPosts: # 填充训练矩阵
#     trainMat.append(setOfWords2Vec(myVocabList,postinDoc))

# p0V,p1V,pAb = trainNB0(trainMat, listClasses)
# # 矩阵中位置对应词汇表单词出现的概率
# print(myVocabList)
# print(p0V) 
# print(p1V)

朴素贝叶斯分类函数

In [60]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # 对应元素相乘，logA*B = logA + logB所以这里是累加
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1) # 向量元素依次相乘
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1) 
    print('p0:', p0)
    print('p1:', p1)
    if p1 > p0:
        return 1
    else:
        return 0


In [61]:
def testingNB():
    # 创建实验样本
    listOPosts, listclasses = loadDataSet()
    # 创建词汇表,将输入文本中的不重复的单词进行提取组成单词向量
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        # 将实验样本向量化若postinDoc中的单词在myVocabList出现则将returnVec该位置的索引置1
        # 将6组数据list存储在trainMat中
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    # 训练朴素贝叶斯分类器
    p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(listclasses))
    # 测试样本1
    testEntry = ['love', 'my', 'dalmation']
    # 测试样本向量化返回这三个单词出现位置的索引
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    if classifyNB(thisDoc, p0V, p1V, pAb):
        # 执行分类并打印结果
        print(testEntry, '属于侮辱类')
    else:
        # 执行分类并打印结果
        print(testEntry, '属于非侮辱类')
    # 测试样本2
    testEntry = ['stupid', 'garbage']
    # 将实验样本向量化
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    if classifyNB(thisDoc, p0V, p1V, pAb):
        # 执行分类并打印结果
        print(testEntry, '属于侮辱类')
    else:
        # 执行分类并打印结果
        print(testEntry, '属于非侮辱类')
    

In [62]:
testingNB()

p0: -7.694848072384611
p1: -9.826714493730215
['love', 'my', 'dalmation'] 属于非侮辱类
p0: -7.20934025660291
p1: -4.702750514326955
['stupid', 'garbage'] 属于侮辱类
